In [170]:
import multiprocessing

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import re

from gensim.models import FastText,KeyedVectors,fasttext

from sklearn import metrics

#import torch
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
from tensorflow.keras.layers import Dense,Activation,Reshape,Conv2D,LeakyReLU,concatenate,Flatten,Reshape
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential,Model

from sklearn.model_selection import train_test_split

In [33]:
def get_faq(urls,question_data,answer_data):
    #
    #  urls - list [urls1,urls2,..]
    #  question_data, answer_data - list [tag_value,class_value]  
    #
    #  return faq - list shape (n,2), where [:,0] - question, [:,1] - answer
    #
    question_tag,question_class=question_data
    answer_tag,answer_class=answer_data
    
    rs =[ requests.get(url) for url in urls]
    responses = [r.text.encode('utf-8') for r in rs]
    soups = [BeautifulSoup(response) for response in responses]

    faq=[]
    for soup in soups:
        question=soup.findAll(question_tag, class_=question_class)
        answer=soup.findAll(answer_tag,class_=answer_class)
        temp_questions=[]
        temp_answers=[]
        for q in question:
            temp_questions.append(q.text)
        for ans in answer:
            temp_answers.append(ans.text)
        
        temp_questions=np.array(temp_questions).reshape((-1,1))
        temp_answers=np.array(temp_answers).reshape((-1,1))
        temp_faq=np.concatenate([temp_questions,temp_answers],axis=1)
        faq.append(temp_faq)


    
    return faq

In [34]:
urls1=['https://www.mosenergosbyt.ru/common/lobby/questions/category_5749.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5739.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48100.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48101.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48102.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48103.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5745.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5763.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5754.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5716.php']

urls2=['https://mosoblgaz.ru/company/query_answer/']

faq1=get_faq(urls1,['button','btn btn-link collapsed'],['div','collapse'])
faq2=get_faq(urls2,['div','faq-list-item__title js-accordion-item-title'],
             ['div','faq-list-item__text-inner js-accordion-item-inner'])


In [35]:
faq1[0][0][1]

'\n\n                    Для переоформления лицевого счета на нового владельца, собственнику необходимо обратиться в любое отделение АО\n                    «Мосэнергосбыт».\n                \n\n                    При себе необходимо иметь следующие документы:\n                \n\nДоговор купли-продажи;\nСвидетельство о государственной регистрации права\xa0собственности\xa0(если\n                        квартира в собственности);\n                    \nКопия финансового лицевого счета из жилищной организации (если квартира муниципальная);\nПаспорт;\nТекущие показания прибора учета на день обращения в отделение АО «Мосэнергосбыт».\n\n\n                    Все документы должны быть представлены в оригинале или копии, заверенные нотариусом.\nТакже оформить заявку на изменения данных с прикреплением необходимых документов можно в Едином личном кабинете физического лица во вкладке «Реквизиты ЛС»/Общая информация/ Изменить, далее в открывшемся списке выбрать «Собственника/нанимателя» при ус

In [ ]:
import tarfile
tar = tarfile.open("test.tar", "r")
tar.extractall()


## Search engine

FastText documents embeddings vs questions

document=questions+answers on one theme

In [36]:
ft_model= fasttext.FastTextKeyedVectors.load('214/model.model')

In [37]:
def sentence_embedding(model,text):
    for sentence in text:
        txt=sentence.split()
        val=0
        for word in txt:
            val+=model[word]

    return val/len(txt)

In [72]:
questions=[]
documents=[]
class_num=len(faq1)
questions2class=dict()
quesions_classes=[]



for i,document in enumerate(faq1):
    
    for line in document:
        question=line[0]
        questions.append(question)
        questions_classes.append(i)
        questions2class[question]=i
        
        
    temp_documents=[]
    for line in document:
        temp_documents.append(line[0])
        temp_documents.append(line[1])

        
    documents.append(temp_documents)




In [73]:
documents_embs=np.zeros((len(documents),300))
for i,document in enumerate(documents):
    documents_embs[i]=sentence_embedding(ft_model,document)
    
questions_embs=np.zeros((len(questions),300))
for i,question in enumerate(questions):
    questions_embs[i]=sentence_embedding(ft_model,[question])
    

In [83]:
true_false_documents=[]
true_questions=[]
true_question_classes=[]

for question_class,question in zip(questions_classes,questions_embs):
    true_false_documents.append(documents_embs[question_class])
    true_questions.append(question)
    true_question_classes.append(question_class)
    
    flag=True
    while flag:
        rnd_index=np.random.randint(0,10)
        if rnd_index!=question_class:
            flag=False
            
    true_false_documents.append(documents_embs[rnd_index])
    true_questions.append(question)
    true_question_classes.append(class_num+1)
        
true_false_documents=np.array(true_false_documents)
true_questions=np.array(true_questions)
true_question_classes=np.array(true_question_classes)
        

In [157]:

np.random.seed(41)

x1_train,x1_test,y1_train,y1_test=train_test_split(true_false_documents,true_question_classes)

y1_train=tf.one_hot(y1_train,depth=class_num+1)

np.random.seed(41)
x2_train,x2_test,y2_train,y2_test=train_test_split(true_questions,true_question_classes)
y2_train=tf.one_hot(y2_train,depth=class_num+1)

In [176]:
def search_head(x):
    x=Dense(300)(x)
    x=Activation(LeakyReLU(0.2))(x)
    x=Reshape((10,10,3))(x)
    x=Conv2D(32,(3,3),padding='same')(x)
    x=Conv2D(64,(3,3),padding='same')(x)
    x=Flatten()(x)
    return x
    

In [177]:
document_input=Input(shape=(300,))
question_input=Input(shape=(300,))

document_output=search_head(tf.expand_dims(document_input,1))
question_output=search_head(tf.expand_dims(question_input,1))

x=concatenate((document_output,question_output),axis=1)
x=Dense(256)(x)
x=Activation(LeakyReLU(0.2))(x)

x=Dense(class_num+1,activation='softmax')(x)

model=Model(inputs=[document_input,question_input],outputs=[x])

In [178]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           [(None, 300)]        0                                            
__________________________________________________________________________________________________
tf.expand_dims_2 (TFOpLambda)   (None, 1, 300)       0           input_35[0][0]                   
__________________________________________________________________________________________________
tf.expand_dims_3 (TFOpLambda)   (None, 1, 300)       0           input_36[0][0]                   
____________________________________________________________________________________________

In [111]:
y1_train.shape

(142,)

In [179]:
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['categorical_crossentropy'])
history=model.fit([x1_train,x2_train],y1_train,verbose=1,batch_size=1,epochs=15)

Epoch 1/15
142/142 [==============================] - 6s 7ms/step - loss: 321.1206 - categorical_crossentropy: 321.1206
Epoch 2/15
142/142 [==============================] - 1s 6ms/step - loss: 17320.4709 - categorical_crossentropy: 17320.4709
Epoch 3/15
142/142 [==============================] - 1s 7ms/step - loss: 217406.6017 - categorical_crossentropy: 217406.6017
Epoch 4/15
142/142 [==============================] - 1s 6ms/step - loss: 827465.1746 - categorical_crossentropy: 827465.1746: 0s - loss: 723005.6580 - categorical_crossen
Epoch 5/15
142/142 [==============================] - 1s 7ms/step - loss: 2221594.8121 - categorical_crossentropy: 2221594.8121
Epoch 6/15
142/142 [==============================] - 1s 6ms/step - loss: 5405401.8837 - categorical_crossentropy: 5405401.8837
Epoch 7/15
142/142 [==============================] - 1s 6ms/step - loss: 12379370.8007 - categorical_crossentropy: 12379370.8007
Epoch 8/15
142/142 [==============================] - 1s 6ms/step - loss

## Paragraph ranker

In [27]:
ft_model.most_similar('сложный')

[('сложнейший', 0.7793360948562622),
 ('трудный', 0.766022801399231),
 ('несложный', 0.7286560535430908),
 ('трудоемкий', 0.7172977328300476),
 ('запутанный', 0.6913840770721436),
 ('специфичный', 0.687846839427948),
 ('важный', 0.6857823729515076),
 ('тривиальный', 0.6841895580291748),
 ('интересный', 0.6826520562171936),
 ('непростой', 0.680992066860199)]

In [125]:
document_emb.shape

(14, 300)

## Question Answering

[Reference](https://medium.com/swlh/fine-tuning-bert-for-text-classification-and-question-answering-using-tensorflow-framework-4d09daeb3330#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjFiZjhhODRkM2VjZDc3ZTlmMmFkNWYwNmZmZDI2MDcwMWRkMDZkOTAiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2MjU5MDYxNDIsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwNjU4MDUxNjE5NzU3NTk0MTk2NCIsImhkIjoibWllbS5oc2UucnUiLCJlbWFpbCI6ImRna2FncmFtYW55YW5AbWllbS5oc2UucnUiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXpwIjoiMjE2Mjk2MDM1ODM0LWsxazZxZTA2MHMydHAyYTJqYW00bGpkY21zMDBzdHRnLmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwibmFtZSI6ItCU0LDQstC40LQg0JrQsNCz0YDQsNC80LDQvdGP0L0iLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EtL0FPaDE0R2hGdEZGSkZMaDlyUFV0QXdZLXlyZWlRczJObjEyQ2xpc3hmX3hQPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6ItCU0LDQstC40LQiLCJmYW1pbHlfbmFtZSI6ItCa0LDQs9GA0LDQvNCw0L3Rj9C9IiwiaWF0IjoxNjI1OTA2NDQyLCJleHAiOjE2MjU5MTAwNDIsImp0aSI6IjhiYmVjYmRjYWRiZTAyZDZkNjQ1MTMxY2Q2NzJhZTI4MWViMTdkNWYifQ.LRr773uwpuGgpYnO7WsltRMVbC3JdFg7DBIlPtnLhN11OpKyNZ5X3y2ZGLx72_tJaSXZS52xdLcEsLuM4Tk1Ta_4ifnuAMkkeJMYJct0DpJOqXGYWS9S2atl8JLvbQaLyNrseNHneLtzRgtj--Htk1Lq0red-VmGxO849tzhKfjfQDPw-PsKxnhKMxHoEHJs91z-djL_L8ATQ6p86TcSWzWGsi4Ya69TmkMiRw-W2eMaZjN11gjSXUCnJKUUC122fluOmDzsGnUbtpoSGD-86mTa368FTFLbQSC7M8SGiZKo_pIZCcKrGjBQM-bJFAMs3-I8J7nM6782Vlh_CZiBew)

In [49]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/distilrubert-base-cased-conversational")

Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
 model.cuda()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [10]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)
# (312,)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(768,)


In [85]:
tokens=tokenizer(['Хочу кредитную карту','Как взять кредит','Хочу взять кредит наличными'],padding=True)

text=['Хочу кредитную карту','Хочу взять кредит наличными',
      'Кредитование','Страхование','Инвестиции']
tokens_vectors=get_embedding(ft_model,text)

In [119]:
masks=np.array(tokens['attention_mask'])
ids=np.array(tokens['input_ids'])
data=[np.concatenate([ids,masks],axis=1)]
data

[array([[  101,  9620, 77624,  5538,   102,     0,     1,     1,     1,
             1,     1,     0],
        [  101,  1235,  3715,  7916,   102,     0,     1,     1,     1,
             1,     1,     0],
        [  101,  9620,  3715,  7916, 49392,   102,     1,     1,     1,
             1,     1,     1]])]

In [117]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [120]:
pt_outputs = model(data, output_hidden_states=True, output_attentions=True)

AttributeError: 'list' object has no attribute 'size'

In [101]:
metrics.pairwise.cosine_similarity(tokens['input_ids'])

array([[1.        , 0.49685953, 0.1073186 ],
       [0.49685953, 1.        , 0.20703129],
       [0.1073186 , 0.20703129, 1.        ]])

In [122]:
from torchsummaryX import summary
arch = summary(model, torch.rand((1, 3, 256, 256)))

RuntimeError: The expanded size of the tensor (256) must match the existing size (3) at non-singleton dimension 3.  Target sizes: [1, 3, 256, 256].  Tensor sizes: [1, 3]

In [71]:
def preprocess(context,questions,answer):
    
    t_context = tokenizer(context,padding=True)
    t_questions = tokenizer(questions,padding=True)
        
    # create inputs as usual
    input_ids = t_context['input_ids'] + t_questions['input_ids']
    token_type_ids = [0] * len(t_context['input_ids']) + t_questions['attention_mask']
    attention_mask = [1] * len(input_ids)

    return input_ids,token_type_ids,attention_mask

In [72]:
questions=list(faq1[:,1].reshape((1,-1))[0])
context=" ".join(questions)
input_ids,token_type_ids,attention_mask=preprocess(context,questions,0)

In [84]:
len(context.split())

7289

In [94]:
string=questions[0].replace('\n','')
len(string.replace('/','').split())

106

In [ ]:
class Sample:
    def __init__(self, question, context, start_char_idx=None, answer_text=None, all_answers=None):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False
        self.start_token_idx = -1
        self.end_token_idx = -1

    def preprocess(self):
        # clean context and question
        context = " ".join(str(self.context).split())
        question = " ".join(str(self.question).split())
        # tokenize context and question
        tokenized_context = tokenizer.encode(context)
        tokenized_question = tokenizer.encode(question)
        
        # if this is validation or training sample, preprocess answer
        if self.answer_text is not None:
            answer = " ".join(str(self.answer_text).split())
            # check if end character index is in the context
            end_char_idx = self.start_char_idx + len(answer)
            if end_char_idx >= len(context):
                self.skip = True
                return
            # mark all the character indexes in context that are also in answer     
            is_char_in_ans = [0] * len(context)
            for idx in range(self.start_char_idx, end_char_idx):
                is_char_in_ans[idx] = 1
            ans_token_idx = []
            # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context.offsets):
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                self.skip = True
                return
            # get start and end token indexes
            self.start_token_idx = ans_token_idx[0]
            self.end_token_idx = ans_token_idx[-1]
        # create inputs as usual
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            self.skip = True
            return
        self.input_word_ids = input_ids
        self.input_type_ids = token_type_ids
        self.input_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets

In [ ]:
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                if "answers" in qa:
                    answer_text = qa["answers"][0]["text"]
                    all_answers = [_["text"] for _ in qa["answers"]]
                    start_char_idx = qa["answers"][0]["answer_start"]
                    squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
                else:
                    squad_eg = Sample(question, context)
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_word_ids": [],
        "input_type_ids": [],
        "input_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    x = [dataset_dict["input_word_ids"],
         dataset_dict["input_mask"],
         dataset_dict["input_type_ids"]]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

In [ ]:
class ValidationCallback(keras.callbacks.Callback):

    def normalize_text(self, text):
        # convert to lower case
        text = text.lower()
        # remove redundant whitespaces
        text = "".join(ch for ch in text if ch not in set(string.punctuation))
        # remove articles
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        text = re.sub(regex, " ", text)
        text = " ".join(text.split())
        return text

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        # get the offsets of the first and last tokens of predicted answers
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        # for every pair of offsets
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            # take the required Sample object with the ground-truth answers in it
            squad_eg = eval_examples_no_skip[idx]
            # use offsets to get back the span of text corresponding to
            # our predicted first and last tokens
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]
            normalized_pred_ans = self.normalize_text(pred_ans)
            # clean the real answers
            normalized_true_ans = [self.normalize_text(_) for _ in squad_eg.all_answers]
            # check if the predicted answer is in an array of the ground-truth answers
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch + 1}, exact match score={acc:.2f}")

In [ ]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")

with open(train_path) as f:
    raw_train_data = json.load(f)
    
with open(eval_path) as f:
    raw_eval_data = json.load(f)

max_seq_length = 384

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy().decode("utf-8")
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertWordPieceTokenizer(vocab=vocab_file, lowercase=True)
train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")
eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")
start_logits = layers.Dense(1, name="start_logit", use_bias=False)(sequence_output)
start_logits = layers.Flatten()(start_logits)
end_logits = layers.Dense(1, name="end_logit", use_bias=False)(sequence_output)
end_logits = layers.Flatten()(end_logits)
start_probs = layers.Activation(keras.activations.softmax)(start_logits)
end_probs = layers.Activation(keras.activations.softmax)(end_logits)
model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=[start_probs, end_probs])
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer=optimizer, loss=[loss, loss])
model.summary()
model.fit(x_train, y_train, epochs=2, batch_size=8, callbacks=[ValidationCallback(x_eval, y_eval)])
model.save_weights("./weights.h5")

In [ ]:
data = {"data":
    [
        {"title": "Project Apollo",
         "paragraphs": [
             {
                 "context": "The Apollo program, also known as Project Apollo, was the third United States human "
                            "spaceflight program carried out by the National Aeronautics and Space Administration ("
                            "NASA), which accomplished landing the first humans on the Moon from 1969 to 1972. First "
                            "conceived during Dwight D. Eisenhower's administration as a three-man spacecraft to "
                            "follow the one-man Project Mercury which put the first Americans in space, Apollo was "
                            "later dedicated to President John F. Kennedy's national goal of landing a man on the "
                            "Moon and returning him safely to the Earth by the end of the 1960s, which he proposed in "
                            "a May 25, 1961, address to Congress. Project Mercury was followed by the two-man Project "
                            "Gemini. The first manned flight of Apollo was in 1968. Apollo ran from 1961 to 1972, "
                            "and was supported by the two man Gemini program which ran concurrently with it from 1962 "
                            "to 1966. Gemini missions developed some of the space travel techniques that were "
                            "necessary for the success of the Apollo missions. Apollo used Saturn family rockets as "
                            "launch vehicles. Apollo/Saturn vehicles were also used for an Apollo Applications "
                            "Program, which consisted of Skylab, a space station that supported three manned missions "
                            "in 1973-74, and the Apollo-Soyuz Test Project, a joint Earth orbit mission with the "
                            "Soviet Union in 1975.",
                 "qas": [
                     {"question": "What project put the first Americans into space?",
                      "id": "Q1"
                      },
                     {"question": "What program was created to carry out these projects and missions?",
                      "id": "Q2"
                      },
                     {"question": "What year did the first manned Apollo flight occur?",
                      "id": "Q3"
                      },
                     {"question": "What President is credited with the original notion of putting Americans in space?",
                      "id": "Q4"
                      },
                     {"question": "Who did the U.S. collaborate with on an Earth orbit mission in 1975?",
                      "id": "Q5"
                      },
                     {"question": "How long did Project Apollo run?",
                      "id": "Q6"
                      },
                     {"question": "What program helped develop space travel techniques that Project Apollo used?",
                      "id": "Q7"
                      },
                     {"question": "What space station supported three manned missions in 1973-1974?",
                      "id": "Q8"
                      }
                 ]}]}]}

test_samples = create_squad_examples(data)
x_test, _ = create_inputs_targets(test_samples)
pred_start, pred_end = model.predict(x_test)
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
    test_sample = test_samples[idx]
    offsets = test_sample.context_token_to_char
    start = np.argmax(start)
    end = np.argmax(end)
    pred_ans = None
    if start >= len(offsets):
        continue
    pred_char_start = offsets[start][0]
    if end < len(offsets):
        pred_ans = test_sample.context[pred_char_start:offsets[end][1]]
    else:
        pred_ans = test_sample.context[pred_char_start:]
    print("Q: " + test_sample.question)
    print("A: " + pred_ans)